<a href="https://colab.research.google.com/github/Troyanovsky/awesome-TTS-Colab/blob/main/kokoro_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🗣️ Kokoro TTS Google Colab

## 📄 Description  
This Colab notebook uses Kokoro TTS to generate voice audio from text.

**Languages supported**: American English (a), British English (b), Spanish (es), French (fr-fr), Hindi (hi), Italian (it), Japanese (ja), Brazilian Portuguese (pt-br), Mandarin Chinese (zh).


**Capabilities**: Text-to-speech, Multi-lingual, Predefined Voices

---

## How to use
- Follow the instructions to input text to generate and adjust params.
- Run all cells and output will be in `output.wav`

---

## 🔗 Resources

- **GitHub Repository:** [hexgrad/kokoro](https://github.com/hexgrad/kokoro)
- **Model Availability:** [hexgrad/Kokoro-82M](https://huggingface.co/hexgrad/Kokoro-82M)

---

## 🎙️ Explore More TTS Models  
Want to try out additional TTS models? Check out the curated collection here:  
👉 [awesome-TTS-Colab](https://github.com/Troyanovsky/awesome-TTS-Colab)


In [ ]:
!pip install kokoro==0.9.4 soundfile misaki[en] misaki[zh] misaki[ja] pydub
!apt-get -qq -y install espeak-ng > /dev/null 2>&1
!apt-get install -y ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of spacy-curated-transformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.0/698.0 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:

en_us_voices = [
  "af_heart", "af_alloy", "af_aoede", "af_bella", "af_jessica", "af_kore",
  "af_nicole", "af_nova", "af_river", "af_sarah", "af_sky",
  "am_adam", "am_echo", "am_eric", "am_fenrir", "am_liam", "am_michael",
  "am_onyx", "am_puck", "am_santa"
]

en_gb_voices = [
  "bf_emma", "bf_isabella", "bf_alice", "bf_lily",
  "bm_george", "bm_lewis", "bm_daniel", "bm_fable"
]

ja_voices = [
  "jf_alpha", "jf_gongitsune", "jf_nezumi", "jf_tebukuro",
  "jm_kumo"
]

zh_voices = [
  "zf_xiaobei", "zf_xiaoni", "zf_xiaoxiao", "zf_xiaoyi",
  "zm_yunjian", "zm_yunxi", "zm_yunxia", "zm_yunyang"
]

es_voices = [
  "ef_dora", "em_alex", "em_santa", "ff_siwis"
]

hi_voices = [
  "hf_alpha", "hf_beta", "hm_omega", "hm_psi"
]

it_voices = [
  "if_sara", "im_nicola"
]

pt_br_voices = [
  "pf_dora", "pm_alex", "pm_santa"
]

In [ ]:
language = "a"   # Make sure lang_code matches the voice
# 🇺🇸 'a' => American English,
# 🇬🇧 'b' => British English
# 🇪🇸 'e' => Spanish es
# 🇫🇷 'f' => French fr-fr
# 🇮🇳 'h' => Hindi hi
# 🇮🇹 'i' => Italian it
# 🇯🇵 'j' => Japanese
# 🇧🇷 'p' => Brazilian Portuguese pt-br
# 🇨🇳 'z' => Mandarin Chinese
text_to_generate = "This is voice generated by Kokoro"
voice = en_us_voices[0]   # Make sure voice matches the language code

In [ ]:
from IPython.display import Audio, display
from pydub import AudioSegment
import soundfile as sf
import os
from kokoro import KPipeline
import torch

# Ensure pydub can find ffmpeg
from pydub.utils import which
AudioSegment.converter = which("ffmpeg")

pipeline = KPipeline(lang_code=language)

# Directory to save intermediate audio files
output_dir = "audio_fragments"
os.makedirs(output_dir, exist_ok=True)

# Generating audio fragments
audio_files = []
generator = pipeline(text_to_generate, voice=voice, speed=1, split_pattern=r'\n+')

for i, (gs, ps, audio) in enumerate(generator):
    print(f"Fragment {i}:")
    print(f"Graphemes: {gs}")
    print(f"Phonemes: {ps}")

    # Save each fragment as a WAV file
    file_path = f"{output_dir}/{i}.wav"
    sf.write(file_path, audio, 24000)
    audio_files.append(file_path)

# Concatenate all audio fragments
combined = AudioSegment.silent(duration=0)  # Start with a silent segment

for file in audio_files:
    segment = AudioSegment.from_wav(file)
    combined += segment

# Save the combined audio as output.wav
combined_output_path = "output.wav"
combined.export(combined_output_path, format="wav")

# Display the concatenated audio file
display(Audio(combined_output_path, autoplay=True))